In [ ]:
import numpy as np
from PIL import Image, ImageFilter

In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.filter(ImageFilter.GaussianBlur(radius=2))  # Apply Gaussian blur for noise reduction
    return image


In [ ]:
def histogram_equalization(image):
    histogram, bins = np.histogram(image.flatten(), 256, density=True)
    cumulative_distribution = histogram.cumsum()
    cumulative_distribution = 255 * cumulative_distribution / cumulative_distribution[-1]

    equalized_image = np.interp(image, bins[:-1], cumulative_distribution)
    return equalized_image.astype(np.uint8)


In [ ]:
def sobel_filter(image):
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2], 
                        [-1, 0, 1]])    
    sobel_y = np.array([[1, 2, 1],
                        [0, 0, 0],
                        [-1, -2, -1]])

    gradient_x = np.abs(image.convolve(ImageFilter.Kernel((3, 3), sobel_x)))
    gradient_y = np.abs(image.convolve(ImageFilter.Kernel((3, 3), sobel_y)))

    gradient_magnitude = np.sqrt(np.square(gradient_x) + np.square(gradient_y))
    return gradient_magnitude.astype(np.uint8)


In [ ]:
def binarize_image(image, threshold):
    return image > threshold

In [ ]:
def pattern_matching(image, pattern):
    image_width, image_height = image.size
    pattern_width, pattern_height = pattern.size

    for x in range(image_width - pattern_width):
        for y in range(image_height - pattern_height):
            region = image.crop((x, y, x + pattern_width, y + pattern_height))
            difference = np.abs(np.array(region) - np.array(pattern))
            if np.mean(difference) < 20:  # Adjust the threshold as per your requirements
                return True

    return False


In [ ]:
def detect_coins(image_path):
    image = preprocess_image(image_path)
    equalized_image = histogram_equalization(image)
    gradient_magnitude = sobel_filter(equalized_image)
    binary_image = binarize_image(gradient_magnitude, threshold=100)

    # Load the pattern image of a coin to match against
    coin_pattern = Image.open("coin_pattern.png").convert("L")

    if pattern_matching(binary_image, coin_pattern):
        print("Coin detected!")
    else:
        print("No coin detected.")